In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb


x_b_train = pd.read_csv('data_b_train.csv')
x_b_test = pd.read_csv('data_b_test.csv')
x_m_train = pd.read_csv('data_m_train.csv')
x_m_test = pd.read_csv('data_m_test.csv')

y_train = pd.read_csv('y_train.csv')


# b表使用特征
b_col = ['id', 'x_num_0', 'x_num_1', 'x_num_2', 'x_num_3', 'x_cat_0', 'x_cat_1',
           'x_cat_2', 'x_cat_3', 'x_cat_4', 'x_cat_9', 'x_cat_10', 'x_cat_11']

x_b_train = x_b_train[b_col]
x_b_test = x_b_test[b_col]


m_col = ['id','x_num_70', 'x_num_5', 'x_num_6', 'x_num_7', 'x_num_10',
         'x_num_13', 'x_num_14', 'x_num_15', 'x_num_17', 'x_num_18', 'x_num_20',
         'x_num_22', 'x_num_23', 'x_num_24', 'x_num_25', 'x_num_26', 'x_num_27',
         'x_num_28', 'x_num_30','x_num_32', 'x_num_33',
         'x_num_39', 'x_num_41', 'x_num_43', 'x_num_45', 'x_num_46',
         'x_num_47', 'x_num_49', 'x_num_50', 'x_num_51', 'x_num_55', 
         'x_num_63', 'x_num_68', 'x_num_69']

# x_m_train = x_m_train[m_col]
# x_m_test = x_m_test[m_col]


x_m_test.replace(-99.0, np.nan, inplace=True)
x_b_test.replace(-99,0,inplace=True)

m_cols = [ 'x_num_70', 'x_num_5', 'x_num_6', 'x_num_7', 'x_num_10',
          'x_num_14', 'x_num_15', 'x_num_17', 'x_num_18', 'x_num_20',
          'x_num_22', 'x_num_24', 'x_num_25', 'x_num_26', 'x_num_27',
          'x_num_28', 'x_num_30','x_num_32', 'x_num_33','x_num_41', 'x_num_43', 
          'x_num_45', 'x_num_46','x_num_49', 'x_num_50','x_num_55', 
           'x_num_63', 'x_num_68', 'x_num_69']



# 选取相同id下最大的timestamp,最大值，标准差，离散系数

def changeName(df, name):
    new = []
    for col in df.columns.values:
        if col is 'id':
            new.append(col)
        else:
            new.append(col+'_'+name)
    return new
    


x_m_train_mean = x_m_train[m_col].groupby('id', as_index=False).mean()
x_m_train_time = x_m_train[m_col+['timestamp']].sort_values('timestamp', ascending=False).groupby('id', as_index=False).first()
x_m_train_time.drop('timestamp', axis=1, inplace=True)
x_m_train_max = x_m_train[m_col].groupby('id', as_index=False).max()
x_m_train_std = x_m_train[m_col].groupby('id', as_index=False).std()
x_m_train_cov = x_m_train_std / x_m_train_mean
x_m_train_median = x_m_train[m_col].groupby('id', as_index=False).median()


# time/max
# x_m_train_div = x_m_train_time / x_m_train_max


# # 计算差值的std
# x_m_train_diff = x_m_train[m_col].groupby('id', as_index=False).diff()
# x_m_train_diff['id'] = x_m_train['id']
# x_m_train_diff_min = x_m_train_diff.groupby('id', as_index=False).min()
# x_m_train_diff_std = x_m_train_diff.groupby('id', as_index=False).std()



x_m_train_time.columns = changeName(x_m_train_time, 'time')
x_m_train_max.columns = changeName(x_m_train_max, 'max')
x_m_train_std.columns = changeName(x_m_train_std, 'std')
x_m_train_std['id']=x_m_train_max['id']
x_m_train_cov.columns = changeName(x_m_train_cov, 'cov')
x_m_train_cov['id']=x_m_train_max['id']
x_m_train_median.columns = changeName(x_m_train_median, 'median')
# x_m_train_diff_std.columns = changeName(x_m_train_diff_std, 'diff_std')

# x_m_train_div.columns = changeName(x_m_train_div, 'div')
# x_m_train_div['id']=x_m_train_max['id']

    
x_m_train = pd.merge(x_m_train_time, x_m_train_max, how='left',on='id')
x_m_train = pd.merge(x_m_train, x_m_train_std, how='left',on='id')
x_m_train = pd.merge(x_m_train, x_m_train_cov, how='left',on='id')
x_m_train = pd.merge(x_m_train, x_m_train_median, how='left',on='id')



x_m_test_mean = x_m_test[m_col].groupby('id', as_index=False).mean()
x_m_test_time = x_m_test[m_col+['timestamp']].sort_values('timestamp', ascending=False).groupby('id', as_index=False).first()
x_m_test_time.drop('timestamp', axis=1, inplace=True)
x_m_test_max = x_m_test[m_col].groupby('id', as_index=False).max()
x_m_test_std = x_m_test[m_col].groupby('id', as_index=False).std()
x_m_test_cov = x_m_test_std / x_m_test_mean
x_m_test_median = x_m_test[m_col].groupby('id', as_index=False).median()


# time/max
# x_m_test_div = x_m_test_time / x_m_test_max
# # 计算差值的std
# x_m_test_diff = x_m_test[m_col].groupby('id', as_index=False).diff()
# x_m_test_diff['id'] = x_m_test['id']
# x_m_test_diff_min = x_m_test_diff.groupby('id', as_index=False).min()
# x_m_test_diff_std = x_m_test_diff.groupby('id', as_index=False).std()
# x_m_train_diff_std['id'] = x_m_train_max['id']


x_m_test_time.columns = changeName(x_m_test_time, 'time')
x_m_test_max.columns = changeName(x_m_test_max, 'max')
x_m_test_std.columns = changeName(x_m_test_std, 'std')
x_m_test_std['id']=x_m_test_max['id']
x_m_test_cov.columns = changeName(x_m_test_cov, 'cov')
x_m_test_cov['id']=x_m_test_max['id']
x_m_test_median.columns = changeName(x_m_test_median, 'median')
# x_m_test_diff_std.columns = changeName(x_m_test_diff_std, 'diff_std')

# x_m_test_div.columns = changeName(x_m_test_div, 'div')
# x_m_test_div['id']=x_m_test_max['id']


    
x_m_test = pd.merge(x_m_test_time, x_m_test_max, how='left',on='id')  
x_m_test = pd.merge(x_m_test, x_m_test_std, how='left',on='id')
x_m_test = pd.merge(x_m_test, x_m_test_cov, how='left',on='id')
x_m_test = pd.merge(x_m_test, x_m_test_median, how='left',on='id')


# 将b表与m表拼接
train = pd.merge(x_b_train, x_m_train, how='left',on='id')
test = pd.merge(x_b_test, x_m_test, how='left',on='id')


y_train = y_train['target']

x_train = train.drop('id', axis=1)
x_test = test.drop('id', axis=1)



# # 构建模型
# train_data = lgb.Dataset(x_train, label=y_train)

# params = {'boosting':'gbdt',
#           'objective':'binary',
#           'max_depth':6,
#           'learning_rate':0.01,
#           'num_leaves':50,
#           'metric': ['binary_logloss', 'auc']
#          }


# my_model = lgb.train(params, train_data, 500)

In [6]:
# params = {
#              'objective':'binary',
#              'is_unbalance': True,
#              'metric': 'binary_logloss,auc',
#              'max_depth':6,
#              'num_leaves':150,
#              'learning_rate':0.1,
#             'feature_fraction':0.7,
#              'min_child_samples':21,
#              'min_child_weight':0.001,
#              'bagging_fraction' :1,
#              'bagging_freq':2,
#              'reg_alpha':0.001,
#              'reg_lambda':8,
#              'cat_smooth': 0,
  
#     }

params = {'boosting':'gbdt',
          'objective':'binary',
          'max_depth':6,
          'learning_rate':0.1,
          'num_leaves':50,
          'metric':['auc', 'binary_logloss']
         }

data_train = lgb.Dataset(x_train, y_train, silent=True)
cv_results = lgb.cv(
    params, lgb.Dataset(x_train, y_train, silent=True), 
    num_boost_round=5000, nfold=5, stratified=False, shuffle=True, metrics=['auc', 'binary_logloss'],
    early_stopping_rounds=50, verbose_eval=50, show_stdv=True, seed=0)

print('best n_estimators:', len(cv_results['auc-mean']), len(cv_results['binary_logloss-mean']))
print('best cv score:', cv_results['auc-mean'][-1], cv_results['binary_logloss-mean'][-1])

# best n_estimators: 526 526
# best cv score: 0.834260740650152 0.258253590504551

[50]	cv_agg's auc: 0.852781 + 0.00480528	cv_agg's binary_logloss: 0.249676 + 0.0119209
[100]	cv_agg's auc: 0.851758 + 0.00449059	cv_agg's binary_logloss: 0.252858 + 0.0119128
best n_estimators: 52 52
best cv score: 0.8530912982651735 0.24948749057736727


In [4]:
from lightgbm import LGBMClassifier

In [7]:
# 寻找最优参数'max_depth'，  'num_leaves'
from sklearn.metrics import roc_auc_score, make_scorer
from sklearn.model_selection import GridSearchCV


params_test1 = {
              'max_depth': range(3, 8, 1),
              'num_leaves':range(20, 150, 10),
             }

# model_lgb = LGBMClassifier(
#                         objective = 'binary',
#                          is_unbalance = True,
#                          metric = 'binary_logloss,auc',
#                          max_depth = 6,
#                          num_leaves = 40,
#                          learning_rate = 0.1,
#                          feature_fraction = 0.7,
#                          min_child_samples=21,
#                          min_child_weight=0.001,
#                          bagging_fraction = 1,
#                          bagging_freq = 2,
#                          reg_alpha = 0.001,
#                          reg_lambda = 8,
#                          cat_smooth = 0,
#                          num_iterations = 1000)

model_lgb = LGBMClassifier(boosting='gbdt',
          objective='binary',
          max_depth=6,
          learning_rate=0.1,
          max_bin=200,
          num_leaves=50,
          metric=['auc', 'binary_logloss'],
                         )


# 创建网格
gsearch1 = GridSearchCV(estimator=model_lgb, param_grid=params_test1, cv = 5, verbose=1, n_jobs=4, scoring= 'roc_auc')

#训练
#打印最优的分类参数
gsearch1.fit(x_train, y_train)
gsearch1.best_params_, gsearch1.best_score_

# 50迭代
# LGBMClassifier(boosting_type='gbdt', class_weight='balanced',
#                colsample_bytree=0.8, importance_type='split', learning_rate=0.1,
#                max_depth=4, min_child_samples=20, min_child_weight=0.001,
#                min_split_gain=0.0, n_estimators=100, n_jobs=-1, num_leaves=50,
#                objective='binary', random_state=None, reg_alpha=0.0,
#                reg_lambda=0.0, silent=True, subsample=0.8,
#                subsample_for_bin=200000, subsample_freq=0)
# 0.8048743175850636


# 2103迭代 25min
# ({'max_depth': 4, 'num_leaves': 50}, 0.7558331524967551)

Fitting 5 folds for each of 65 candidates, totalling 325 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:   11.5s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:  1.1min
[Parallel(n_jobs=4)]: Done 325 out of 325 | elapsed:  2.5min finished


({'max_depth': 5, 'num_leaves': 40}, 0.8548492088809312)

In [9]:
# 寻找最优参数'max_depth'，  'num_leaves'
from sklearn.metrics import roc_auc_score, make_scorer
from sklearn.model_selection import GridSearchCV


params_test1 = {
              'num_leaves': range(30,50,1)
             }

model_lgb = LGBMClassifier(boosting_type='gbdt', objective='binary', num_leaves=40, max_depth=5, 
       learning_rate=0.1, n_estimators=52)
# 创建网格
gsearch1 = GridSearchCV(estimator=model_lgb, param_grid=params_test1, cv = 5, verbose=2, n_jobs=4, scoring= 'roc_auc')

#训练
#打印最优的分类参数
gsearch1.fit(x_train, y_train)
gsearch1.best_params_, gsearch1.best_score_
# ({'max_depth': 9, 'num_leaves': 120}, 0.8171362874665284)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:   10.8s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:   32.3s finished


({'num_leaves': 31}, 0.8520292278361599)

In [10]:
# 寻找最优参数min_data_in_leaf 和 min_sum_hessian_in_leaf
from sklearn.metrics import roc_auc_score, make_scorer
from sklearn.model_selection import GridSearchCV


params_test1={
    'min_child_samples': [8, 9, 10, 11, 12],
    'min_child_weight':[0.001, 0.002]
}

model_lgb = LGBMClassifier(boosting_type='gbdt', objective='binary', num_leaves=31, max_depth=5, 
       learning_rate=0.1, n_estimators=52)
# 创建网格
gsearch1 = GridSearchCV(estimator=model_lgb, param_grid=params_test1, cv = 5, verbose=1, n_jobs=4, scoring= 'roc_auc')

#训练
#打印最优的分类参数
gsearch1.fit(x_train, y_train)
gsearch1.best_params_, gsearch1.best_score_
# ({'min_child_samples': 19, 'min_child_weight': 0.001}, 0.816678716420489)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:   14.0s
[Parallel(n_jobs=4)]: Done  50 out of  50 | elapsed:   16.3s finished


({'min_child_samples': 12, 'min_child_weight': 0.001}, 0.8518072974394683)

In [11]:
# 寻找最优参数 feature_fraction 和 bagging_fraction
from sklearn.metrics import roc_auc_score, make_scorer
from sklearn.model_selection import GridSearchCV


params_test1={
    'feature_fraction': [0.5, 0.6, 0.7, 0.8, 0.9],
    'bagging_fraction': [0.6, 0.7, 0.8, 0.9, 1.0]
}

model_lgb = LGBMClassifier(boosting_type='gbdt', objective='binary', num_leaves=31, max_depth=5, 
       learning_rate=0.1, n_estimators=52,
                           min_child_samples=12,min_child_weight=0.001)
# 创建网格
gsearch1 = GridSearchCV(estimator=model_lgb, param_grid=params_test1, cv = 5, verbose=1, n_jobs=4, scoring= 'roc_auc')

#训练
#打印最优的分类参数
gsearch1.fit(x_train, y_train)
gsearch1.best_params_, gsearch1.best_score_
# ({'bagging_fraction': 0.6, 'feature_fraction': 0.5}, 0.8185608706918892)

Fitting 5 folds for each of 25 candidates, totalling 125 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:   10.6s
[Parallel(n_jobs=4)]: Done 125 out of 125 | elapsed:   30.5s finished


({'bagging_fraction': 0.6, 'feature_fraction': 0.5}, 0.8542423642508912)

In [10]:
# 寻找最优正则化参数
from sklearn.metrics import roc_auc_score, make_scorer
from sklearn.model_selection import GridSearchCV


params_test1={
    'reg_alpha': [0, 0.08, 0.3, 0.5],
    'reg_lambda': [0, 0.08, 0.3, 0.5]
}

model_lgb = LGBMClassifier(boosting_type='gbdt', objective='binary', num_leaves=50, max_depth=6, 
       class_weight='balanced',learning_rate=0.01, n_estimators=500, min_data_in_leaf=10, max_bin=200,
                           min_child_samples=8,min_child_weight=0.001,
                          bagging_fraction=0.6, feature_fraction=0.7)
# 创建网格
gsearch1 = GridSearchCV(estimator=model_lgb, param_grid=params_test1, cv = 5, verbose=1, n_jobs=2, scoring= 'roc_auc')

#训练
#打印最优的分类参数
gsearch1.fit(x_train, y_train)
gsearch1.best_params_, gsearch1.best_score_
# ({'reg_alpha': 0, 'reg_lambda': 0.3}, 0.8201618208516888)

Fitting 5 folds for each of 16 candidates, totalling 80 fits


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:   26.0s
[Parallel(n_jobs=2)]: Done  80 out of  80 | elapsed:   43.5s finished


({'reg_alpha': 0.5, 'reg_lambda': 0.5}, 0.835574640173394)

In [11]:
params_test1={
   'cat_smooth': [0,10,20],
}

model_lgb = LGBMClassifier(boosting_type='gbdt', objective='binary', num_leaves=50, max_depth=6, 
       class_weight='balanced',learning_rate=0.01, n_estimators=500, min_data_in_leaf=10, max_bin=200,
                           min_child_samples=8,min_child_weight=0.001,
                          bagging_fraction=0.6, feature_fraction=0.7,
                          reg_alpha=0.5, reg_lambda= 0.5)
# 创建网格
gsearch1 = GridSearchCV(estimator=model_lgb, param_grid=params_test1, cv = 5, verbose=1, n_jobs=2, scoring= 'roc_auc')

#训练
#打印最优的分类参数
gsearch1.fit(x_train, y_train)
gsearch1.best_params_, gsearch1.best_score_
# ({'reg_alpha': 0, 'reg_lambda': 0.3}, 0.8201618208516888)

Fitting 5 folds for each of 3 candidates, totalling 15 fits


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  15 out of  15 | elapsed:    8.4s finished


({'cat_smooth': 0}, 0.835574640173394)

In [12]:
params = {'boosting':'gbdt',
          'objective':'binary',
          'max_depth':6,
          'learning_rate':.01,
          'max_bin':200,
          'num_leaves':150,
          'min_data_in_leaf': 10,
          'class_weight':'balanced',
          'metric':['auc', 'binary_logloss'],
          'in_data_in_leaf':10, 
          'max_bin':200,
        'min_child_samples':8,
          'min_child_weight':0.001,
          'bagging_fraction':0.6, 
          'feature_fraction':0.7,
          'reg_alpha':0.5,
          'reg_lambda':0.5,
          'cat_smooth': 0
         }

data_train = lgb.Dataset(x_train, y_train, silent=True)
cv_results = lgb.cv(
    params, lgb.Dataset(x_train, y_train, silent=True), 
    num_boost_round=5000, nfold=5, stratified=False, shuffle=True, metrics=['auc', 'binary_logloss'],
    early_stopping_rounds=50, verbose_eval=50, show_stdv=True, seed=0)

print('best n_estimators:', len(cv_results['auc-mean']), len(cv_results['binary_logloss-mean']))
print('best cv score:', cv_results['auc-mean'][-1], cv_results['binary_logloss-mean'][-1])

[50]	cv_agg's auc: 0.827779 + 0.00959428	cv_agg's binary_logloss: 0.293895 + 0.00858494
[100]	cv_agg's auc: 0.830379 + 0.00727895	cv_agg's binary_logloss: 0.27655 + 0.00858205
[150]	cv_agg's auc: 0.832989 + 0.00778029	cv_agg's binary_logloss: 0.267693 + 0.00883868
[200]	cv_agg's auc: 0.834474 + 0.00674148	cv_agg's binary_logloss: 0.262646 + 0.00866591
[250]	cv_agg's auc: 0.835437 + 0.00669463	cv_agg's binary_logloss: 0.259918 + 0.00872479
[300]	cv_agg's auc: 0.836423 + 0.00694939	cv_agg's binary_logloss: 0.258202 + 0.00896507
[350]	cv_agg's auc: 0.836869 + 0.00720624	cv_agg's binary_logloss: 0.257276 + 0.00917744
[400]	cv_agg's auc: 0.837424 + 0.00726049	cv_agg's binary_logloss: 0.256635 + 0.009459
[450]	cv_agg's auc: 0.837672 + 0.0072056	cv_agg's binary_logloss: 0.256288 + 0.00963297
[500]	cv_agg's auc: 0.837777 + 0.00736242	cv_agg's binary_logloss: 0.256207 + 0.00982969
[550]	cv_agg's auc: 0.837849 + 0.00760028	cv_agg's binary_logloss: 0.256144 + 0.0100755
[600]	cv_agg's auc: 0.83789